# Project description

The following project is the final assigment of the <b>SQL for Data Science</b> course of the <b>University of California, Davis</b>. based on datasets sourced from the <b>United States Department of Agriculture's (USDA) open data portal</b>. First, the CSV files will be uploaded into SQL tables, and then several queries will be performed to retrieve the necessary data to answer the required questions.

<b><u>Scenario</u></b>:

Data Scientist at USDA (United States Department of Agriculture)

<b><u>Context</u></b>:

<u>The datasets include</u>:

milk_production, cheese_production, coffee_production, honey_production, yogurt_production, and a state_lookup table.

The data spans multiple years and states, with varying levels of production for each commodity.

<u>Objectives</u>:

Assess state-by-state production for each commodity.

Identify trends or anomalies.

Offer data-backed suggestions for areas that may need more attention.

## Libraries and custom functions

In [403]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [404]:
def my_query(query):
    # Execute the query
    cursor.execute(query)
    
    # Fetch all results from the executed query
    results = cursor.fetchall()
    
    # Print the results
    for row in results:
        print(row)

## Tables uploading

### Importing files

In [405]:
directory = '/home/martin/Documents/sql_course'
files = os.listdir(directory)
files = []
for f in os.listdir(directory):
    full_path = os.path.join(directory, f)
    if os.path.isfile(full_path) and full_path.endswith('csv'):
        files.append(full_path)                       

In [406]:
tables = {}
for f in files:
    name = f.split('/')[-1].split('.')[0]
    tables[name] = pd.read_csv(f)

In [407]:
for table in tables.keys():
    print(tables[table].dtypes)
    print()

Year              int64
Period           object
Geo_Level        object
State_ANSI      float64
Commodity_ID      int64
Domain           object
Value            object
dtype: object

Year              int64
Geo_Level        object
State_ANSI      float64
Commodity_ID      int64
Value            object
dtype: object

State         object
State_ANSI     int64
dtype: object

Year             int64
Period          object
Geo_Level       object
State_ANSI       int64
Commodity_ID     int64
Value           object
dtype: object

Year              int64
Period           object
Geo_Level        object
State_ANSI      float64
Commodity_ID      int64
Domain           object
Value            object
dtype: object

Year             int64
Period          object
Geo_Level       object
State_ANSI       int64
Commodity_ID     int64
Domain          object
Value           object
dtype: object

Year              int64
Period           object
Geo_Level        object
State_ANSI      float64
Commodity_ID     

### Preprocessing tables

The column Value has object as dtype thus it needs to be pre-processed before to be incorporated to the SQLite database. as the they have several values that cannot be interpreted as a specific column type. For example, the Value column has a comma as a thousand separator.  

In [408]:
for table in tables.keys():
    if 'Value' in tables[table].columns:
        # deleting thousand separator and transforming to numeric
        tables[table].Value= tables[table].Value.replace(',', '', regex=True)
        tables[table].Value = pd.to_numeric(tables[table].Value, errors='coerce')

In [409]:
for table in tables.keys():
    print(tables[table].dtypes)
    print()

Year              int64
Period           object
Geo_Level        object
State_ANSI      float64
Commodity_ID      int64
Domain           object
Value           float64
dtype: object

Year              int64
Geo_Level        object
State_ANSI      float64
Commodity_ID      int64
Value             int64
dtype: object

State         object
State_ANSI     int64
dtype: object

Year             int64
Period          object
Geo_Level       object
State_ANSI       int64
Commodity_ID     int64
Value            int64
dtype: object

Year              int64
Period           object
Geo_Level        object
State_ANSI      float64
Commodity_ID      int64
Domain           object
Value             int64
dtype: object

Year              int64
Period           object
Geo_Level        object
State_ANSI        int64
Commodity_ID      int64
Domain           object
Value           float64
dtype: object

Year              int64
Period           object
Geo_Level        object
State_ANSI      float64
Commodity_

### Uploding tables to SQL

In [410]:
database = 'sql_course.db'
for table in tables.keys():
    conn = sqlite3.connect(database)
    cursor = conn.cursor()
    tables[table].to_sql(table, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

# Questions

In [411]:
# database connection
database = 'sql_course.db'
conn = sqlite3.connect(database)
cursor = conn.cursor()

## Question 1
Can you find out the total milk production for 2023? Your manager wants this information for the yearly report.

What is the total milk production for 2023?

In [412]:
query = """
SELECT SUM(Value) 
FROM milk_production 
WHERE Year = 2023;
"""
my_query(query)

(91812000000,)


## Question 2

Which states had cheese production greater than 100 million in April 2023? The Cheese Department wants to focus their marketing efforts there. 

How many states are there?

In [413]:
query = """
SELECT COUNT(c.Value)
FROM state_lookup l
INNER JOIN cheese_production c
ON l.State_ANSI = c.State_ANSI
WHERE c.Year = 2023 AND c.Period = 'APR' AND c.Value > 100000000; 
"""

my_query(query)

(2,)


## Question 3

Your manager wants to know how coffee production has changed over the years. 

What is the total value of coffee production for 2011?

In [414]:
query = """
SELECT Year
, SUM(Value) 
FROM coffee_production 
WHERE Year = 2011;
"""
my_query(query)

(2011, 7600000)


## Question 4

There's a meeting with the Honey Council next week. Find the average honey production for 2022 so you're prepared.

In [415]:
query = """
SELECT AVG(Value) 
FROM honey_production 
WHERE Year = 2022;
"""

my_query(query)

(3133275.0,)


## Question 5

The State Relations team wants a list of all states names with their corresponding ANSI codes. Can you generate that list?

What is the State_ANSI code for Florida?

In [416]:
query = """
SELECT * 
FROM state_lookup
WHERE State = 'FLORIDA';
"""

my_query(query)

('FLORIDA', 12)


## Question 6

For a cross-commodity report, can you list all states with their cheese production values, even if they didn't produce any cheese in April of 2023?

What is the total for NEW JERSEY?

In [419]:
query = """
SELECT s.State, 
c.Value
FROM state_lookup s
LEFT JOIN cheese_production c 
ON s.State_ANSI = c.State_ANSI AND c.Year = 2023 AND c.Period = 'APR'
WHERE s.State = 'NEW JERSEY';
"""

my_query(query)

('NEW JERSEY', 4889000.0)


## Question 7

List all states from state_lookup that are missing from milk_production in 2023.

How many states are there?

In [397]:
query = """
SELECT COUNT(l.State_ANSI)
FROM state_lookup l
LEFT JOIN milk_production m
ON l.State_ANSI = m.State_ANSI AND m.Year = 2023
WHERE m.State_ANSI IS NULL;
"""

my_query(query)

(26,)


## Question 8

List all states with their cheese production values, including states that didn't produce any cheese in April 2023.

Did Delaware produce any cheese in April 2023?

In [398]:
query = """
SELECT s.State
, c.Value
FROM state_lookup s
LEFT JOIN cheese_production c 
ON s.State_ANSI = c.State_ANSI AND c.Year = 2023 AND c.Period = 'APR'
WHERE s.State = 'DELAWARE';
"""

my_query(query)

('DELAWARE', None)


## Question 9

Find the average coffee production for all years where the honey production exceeded 1 million.

In [425]:
query = """
SELECT AVG(c.Value) AS avg_coffee_production
FROM coffee_production c
WHERE c.Year IN (
    SELECT h.Year
    FROM honey_production h
    WHERE h.Value > 1000000
);
"""

my_query(query)

(6426666.666666667,)


In [401]:
# Closing connection of cursor
conn.close()